In [1]:
import pandas as pd

import ast

In [2]:
foodRaw_df = pd.read_csv('../../Resources/CSVs/all_food.csv')

foodRaw_df.head()

,Restaurant,Food_Name,Nutrients,Calories
0,Subway,Cheese Pizza,"[{'value': 28, 'attr_id': 203}, {'value': 23, ...",720
1,Subway,Sausage Pizza,"[{'value': 35, 'attr_id': 203}, {'value': 35, ...",860
2,Subway,Meatball Pizza,"[{'value': 37, 'attr_id': 203}, {'value': 33, ...",860
3,Subway,Pepperoni Pizza,"[{'value': 35, 'attr_id': 203}, {'value': 33, ...",840
4,Subway,Bacon Pizza,"[{'value': 37, 'attr_id': 203}, {'value': 31, ...",840


In [4]:
nutrient_map = pd.read_csv('../../Resources/CSVs/Nutritionix API v2 - Full Nutrient USDA Field Mapping.csv')

nutrient_map = nutrient_map.set_index('attr_id')

nutrient_map.head()

,2018 NFP,usda_tag,name,unit,notes,bulk_csv_field,cxh_attribute_name,Unnamed: 8
attr_id,,,,,,,,
203,1.0,PROCNT,Protein,g,NaN,nf_protein,PRO-,NaN
204,1.0,FAT,Total lipid (fat),g,NaN,nf_total_fat,FAT,NaN
205,1.0,CHOCDF,"Carbohydrate, by difference",g,NaN,nf_total_carbohydrate,CHO-,NaN
207,0.0,ASH,Ash,g,NaN,NaN,NaN,NaN
208,1.0,ENERC_KCAL,Energy,kcal,NaN,nf_calories,ENER-,NaN


In [5]:
for index,row in foodRaw_df.iterrows(): #iterate each index and row.
    
    nutrients = ast.literal_eval(row['Nutrients']) #The original column was as string. ast.literal_eval -->interprets "strings, bytes, numbers, tuples, lists, dicts, sets, booleans, and None." and returns it in the right format.. Anything else is an error.
    
    for nutrient in nutrients: #Iterates each of the nutrients dictionaries. 
        
        name = nutrient_map.loc[nutrient['attr_id'],'name'] #locates the name in the nutrient_map csv. 
        
        conversion_dict = { #Converts the columns from grams to calories.
            'Protein' : 4, #To cal
            'Carbohydrate, by difference' : 4, #to cal
            'Total lipid (fat)' : 9, #to cal
            'Vitamin A, IU' : 0.3, #to mcg RAE
            'Fatty acids, total trans' : 10,
            'Fatty acids, total saturated' : 9} 
        
        if name in conversion_dict.keys():
            conversionFactor = conversion_dict[name]
        else:
            conversionFactor = 1
        
        value = nutrient['value'] 
        
       
        if name in foodRaw_df.columns: #if the nutrient is a DataFrame column.
                
            foodRaw_df.loc[index,name] = value * conversionFactor #Adds the value. 
        
        else: #if the nutrient is not in the DataFrame
            foodRaw_df[name] =''
            foodRaw_df.loc[index,name] = value * conversionFactor
        
food_df = foodRaw_df.replace(r"^\s*$", 0, regex = True) #Finds and replace empty cells with 0. empty cell pattern = r"^\s*$"

food_df.head()

,Restaurant,Food_Name,Nutrients,Calories,Protein,Total lipid (fat),"Carbohydrate, by difference",Energy,"Sugars, total","Fiber, total dietary","Calcium, Ca","Iron, Fe","Sodium, Na","Vitamin A, IU","Vitamin C, total ascorbic acid",Cholesterol,"Fatty acids, total trans","Fatty acids, total saturated"
0,Subway,Cheese Pizza,"[{'value': 28, 'attr_id': 203}, {'value': 23, ...",720,112,207.0,388,720,7,4,520,7.2,1530,300.0,9.0,50,5.0,81.0
1,Subway,Sausage Pizza,"[{'value': 35, 'attr_id': 203}, {'value': 35, ...",860,140,315.0,392,860,8,4,650,7.2,1870,300.0,9.0,80,5.0,126.0
2,Subway,Meatball Pizza,"[{'value': 37, 'attr_id': 203}, {'value': 33, ...",860,148,297.0,400,860,7,5,650,8.1,1840,300.0,9.0,80,10.0,126.0
3,Subway,Pepperoni Pizza,"[{'value': 35, 'attr_id': 203}, {'value': 33, ...",840,140,297.0,388,840,8,4,650,7.2,1800,300.0,9.0,70,10.0,117.0
4,Subway,Bacon Pizza,"[{'value': 37, 'attr_id': 203}, {'value': 31, ...",840,148,279.0,392,840,8,4,650,7.2,2060,300.0,9.0,70,5.0,108.0


In [20]:
#DataFrame columns order edit and columns rename.

foodFiltered_df = food_df[['Restaurant',
                           'Food_Name',
                           'Calories',
                           'Protein',
                           'Carbohydrate, by difference',
                           'Fiber, total dietary',
                           'Sugars, total',
                           'Total lipid (fat)',
                           'Fatty acids, total trans',
                           'Fatty acids, total saturated',
                           'Cholesterol',
                           'Calcium, Ca',
                           'Iron, Fe',
                           'Sodium, Na',
                           'Vitamin A, IU',
                          'Vitamin C, total ascorbic acid']]

columns_name = {
    'Protein' : 'Protein (Cal)',
    'Carbohydrate, by difference' : 'Carbohydrate (Cal)',
    'Fiber, total dietary' : 'Fiber (gr)',
    'Sugars, total' : 'Total Sugar (gr)',
    'Total lipid (fat)' : 'Total Fat (Cal)',
    'Cholesterol' : 'Cholesterol (mg)',
    'Calcium, Ca' : 'Calcium (mg)',
    'Iron, Fe' : 'Iron (mg)',
    'Sodium, Na' : 'Sodium (mg)',
    'Vitamin A, IU' : 'Vitamin A (mcg RAE)',
    'Vitamin C, total ascorbic acid' : 'Vitamin C (mg)',
    'Fatty acids, total trans' : 'Total Trans Fat (Cal)',
    'Fatty acids, total saturated' : 'Total Saturated Fat (Cal)'
    
}

foodFiltered_df = foodFiltered_df.rename(columns = columns_name)

foodFiltered_df.head()


,Restaurant,Food_Name,Calories,Protein (Cal),Carbohydrate (Cal),Fiber (gr),Total Sugar (gr),Total Fat (Cal),Total Trans Fat (Cal),Total Saturated Fat (Cal),Cholesterol (mg),Calcium (mg),Iron (mg),Sodium (mg),Vitamin A (mcg RAE),Vitamin C (mg)
0,Subway,Cheese Pizza,720,112,388,4,7,207.0,5.0,81.0,50,520,7.2,1530,300.0,9.0
1,Subway,Sausage Pizza,860,140,392,4,8,315.0,5.0,126.0,80,650,7.2,1870,300.0,9.0
2,Subway,Meatball Pizza,860,148,400,5,7,297.0,10.0,126.0,80,650,8.1,1840,300.0,9.0
3,Subway,Pepperoni Pizza,840,140,388,4,8,297.0,10.0,117.0,70,650,7.2,1800,300.0,9.0
4,Subway,Bacon Pizza,840,148,392,4,8,279.0,5.0,108.0,70,650,7.2,2060,300.0,9.0


In [9]:
#Load kcal requirments from FDA

calorieNeeds_male = pd.read_excel('../../FDA Tables XLSX/Appendix 2. Estimated Calorie Needs -2020 Dietary Guidelines _ health.gov_0002.xlsx', sheet_name = 'Males')

calorieNeeds_male = calorieNeeds_male[calorieNeeds_male['AGE'] >= 14]

calorieNeeds_male = calorieNeeds_male.reset_index(drop= True)

calorieNeeds_male

,AGE,Sedentary,Moderately active,Active
0,14,2000,2400,2800
1,15,2200,2600,3000
2,16,2400,2800,3200
3,17,2400,2800,3200
4,18,2400,2800,3200
...,...,...,...,...
58,72,2000,2200,2600
59,73,2000,2200,2600
60,74,2000,2200,2600
61,75,2000,2200,2600


In [10]:
#Health Parameters Male

maleNutrition14to18_dict = {'Protein (Cal)' : 0.2/3,
                           'Carbohydrate (Cal)' : 0.55/3,
                           'Fiber (gr)' : 30.8/3,
                           'Total Sugar (gr)': 40/3, #https://www.livestrong.com/article/333407-how-many-calories-should-i-eat-at-lunch/
                           'Total lipid (Cal)': 0.3/3,
                           'Total Trans Fat (Cal)': .01/3, #https://medlineplus.gov/ency/patientinstructions/000786.htm#:~:text=You%20should%20limit%20saturated%20fat,or%202%20grams%20per%20day.
                           'Total Saturated Fat (Cal)': 0.1/3,
                           'Cholesterol (mg)' : 250/3, #https://www.healthline.com/health/high-cholesterol/rda#:~:text=Following%20dietary%20guidelines%2C%20doctors%20used,high%20risk%20of%20heart%20disease.
                           'Calcium (mg)': 1300/3,
                           'Iron (mg)': 11/2,
                           'Sodium, (mg)': 2300/3,
                           'Vitamin A (mcg RAE)': 900/3,
                            'Vitamin C (mg)' : 75/3}
    
maleNutrition19to30_dict = {'Protein (Cal)' : 0.225/3,
                           'Carbohydrate (Cal)' : 0.55/3,
                           'Fiber (gr)' : 33.6/3,
                           'Total Sugar (gr)': 40/3, #https://www.livestrong.com/article/333407-how-many-calories-should-i-eat-at-lunch/
                           'Total lipid (Cal)': 0.176/3,
                           'Total Trans Fat (Cal)': .01/3, #https://medlineplus.gov/ency/patientinstructions/000786.htm#:~:text=You%20should%20limit%20saturated%20fat,or%202%20grams%20per%20day.
                           'Total Saturated Fat (Cal)': 0.1/3,
                           'Cholesterol (mg)' : 250/3, #https://www.healthline.com/health/high-cholesterol/rda#:~:text=Following%20dietary%20guidelines%2C%20doctors%20used,high%20risk%20of%20heart%20disease.
                           'Calcium (mg)': 1000/3,
                           'Iron (mg)': 8/2,
                           'Sodium, (mg)': 2300/3,
                           'Vitamin A, IU': 900/3,
                            'Vitamin C (mg)' : 90/3}
    
maleNutrition31to50_dict = {'Protein (Cal)' : 0.225/3,
                           'Carbohydrate (Cal)' : 0.55/3,
                           'Fiber (gr)' : 30.8/3,
                           'Total Sugar (gr)': 40/3, #https://www.livestrong.com/article/333407-how-many-calories-should-i-eat-at-lunch/
                           'Total lipid (Cal)': 0.176/3,
                            'Total Trans Fat (Cal)': .01/3, #https://medlineplus.gov/ency/patientinstructions/000786.htm#:~:text=You%20should%20limit%20saturated%20fat,or%202%20grams%20per%20day.
                           'Total Saturated Fat (Cal)': 0.1/3,
                           'Cholesterol (mg)' : 250/3, #https://www.healthline.com/health/high-cholesterol/rda#:~:text=Following%20dietary%20guidelines%2C%20doctors%20used,high%20risk%20of%20heart%20disease.
                           'Calcium (mg)': 1000/3,
                           'Iron (mg)': 8/2,
                           'Sodium (mg)': 2300/3,
                           'Vitamin A (mcg RAE)': 900/3,
                            'Vitamin C (mg)' : 90/3}
    
maleNutrition51_dict = {'Protein (Cal)' : 0.225/3,
                        'Carbohydrate (Cal)' : 0.55/3,
                        'Fiber (gr)' : 28/3,
                        'Total Sugar (gr)': 40/3, #https://www.livestrong.com/article/333407-how-many-calories-should-i-eat-at-lunch/
                        'Total lipid (Cal)': 0.176/3,
                        'Total Trans Fat (Cal)': .01/3, #https://medlineplus.gov/ency/patientinstructions/000786.htm#:~:text=You%20should%20limit%20saturated%20fat,or%202%20grams%20per%20day.
                        'Total Saturated Fat (Cal)': 0.1/3,
                        'Cholesterol (mg)' : 250/3, #https://www.healthline.com/health/high-cholesterol/rda#:~:text=Following%20dietary%20guidelines%2C%20doctors%20used,high%20risk%20of%20heart%20disease.
                        'Calcium (mg)': 1000/3,
                        'Iron (mg)': 8/2,
                        'Sodium (mg)': 2300/3,
                        'Vitamin A (mcg RAE)': 900/3,
                        'Vitamin C (mg)' : 90/3}
     

In [14]:
#Lists of DataFrames with the information of the required nutrients for each age and lifestyle goup.

maleInfo = []

for lifeStyle in ['Sedentary','Moderately active','Active']:
    
    lifeStyle_df = calorieNeeds_male[['AGE', lifeStyle]]
    
    for index,row in lifeStyle_df.iterrows():

        if row['AGE'] >= 14 & row['AGE'] <= 18 :

            for key in maleNutrition14to18_dict.keys():

                if key not in lifeStyle_df.columns:

                    lifeStyle_df[key] = ''

                    if (key == 'Protein (Cal)') |( key == 'Carbohydrate (Cal)') | (key == 'Total lipid (Cal)')| (key == 'Total Trans Fat (Cal)')|( key == 'Total Saturated Fat (Cal)'):

                        lifeStyle_df.loc[index, key] = round(row[lifeStyle] * maleNutrition14to18_dict[key],2)

                    else:

                        lifeStyle_df.loc[index, key] = round(maleNutrition14to18_dict[key],2)

                else:

                    if (key == 'Protein (Cal)') |( key == 'Carbohydrate (Cal)') | (key == 'Total lipid (Cal)')| (key == 'Total Trans Fat (Cal)')|( key == 'Total Saturated Fat (Cal)'):

                        lifeStyle_df.loc[index, key] = round(row[lifeStyle] * maleNutrition14to18_dict[key],2)

                    else:

                        lifeStyle_df.loc[index, key] = round(maleNutrition14to18_dict[key],2)

        elif row['AGE'] > 19 & row['AGE'] <= 30 :

            for key in maleNutrition19to30_dict.keys():

                    if (key == 'Protein (Cal)') |( key == 'Carbohydrate (Cal)') | (key == 'Total lipid (Cal)')| (key == 'Total Trans Fat (Cal)')|( key == 'Total Saturated Fat (Cal)'):

                        lifeStyle_df.loc[index, key] = round(row[lifeStyle] * maleNutrition19to30_dict[key],2)

                    else:

                        lifeStyle_df.loc[index, key] = round(maleNutrition19to30_dict[key],2)

        elif row['AGE'] > 31 & row['AGE'] <= 50 :

            for key in maleNutrition31to50_dict.keys():

                    if (key == 'Protein (Cal)') |( key == 'Carbohydrate (Cal)') | (key == 'Total lipid (Cal)')| (key == 'Total Trans Fat (Cal)')|( key == 'Total Saturated Fat (Cal)'):

                        lifeStyle_df.loc[index, key] = round(row[lifeStyle] * maleNutrition31to50_dict[key],2)

                    else:

                        lifeStyle_df.loc[index, key] = round(maleNutrition31to50_dict[key],2)

        elif row['AGE'] > 31 & row['AGE'] <= 50 :

            for key in maleNutrition51_dict.keys():

                    if (key == 'Protein (Cal)') |( key == 'Carbohydrate (Cal)') | (key == 'Total lipid (Cal)')| (key == 'Total Trans Fat (Cal)')|( key == 'Total Saturated Fat (Cal)'):

                        lifeStyle_df.loc[index, key] = round(row[lifeStyle] * maleNutrition51_dict[key],2)

                    else:

                        lifeStyle_df.loc[index, key] = round(maleNutrition51_dict[key],2)
                        
    maleInfo.append(lifeStyle_df)
                        
maleInfo[0] #List of DataFrames for males and the intake limits for each meal. 

,AGE,Sedentary,Protein (Cal),Carbohydrate (Cal),Fiber (gr),Total Sugar (gr),Total lipid (Cal),Total Trans Fat (Cal),Total Saturated Fat (Cal),Cholesterol (mg),Calcium (mg),Iron (mg),"Sodium, (mg)",Vitamin A (mcg RAE),Vitamin C (mg)
0,14,2000,133.33,366.67,10.27,13.33,200,6.67,66.67,83.33,433.33,5.5,766.67,300,25
1,15,2200,146.67,403.33,10.27,13.33,220,7.33,73.33,83.33,433.33,5.5,766.67,300,25
2,16,2400,160,440,10.27,13.33,240,8,80,83.33,433.33,5.5,766.67,300,25
3,17,2400,160,440,10.27,13.33,240,8,80,83.33,433.33,5.5,766.67,300,25
4,18,2400,160,440,10.27,13.33,240,8,80,83.33,433.33,5.5,766.67,300,25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,72,2000,133.33,366.67,10.27,13.33,200,6.67,66.67,83.33,433.33,5.5,766.67,300,25
59,73,2000,133.33,366.67,10.27,13.33,200,6.67,66.67,83.33,433.33,5.5,766.67,300,25
60,74,2000,133.33,366.67,10.27,13.33,200,6.67,66.67,83.33,433.33,5.5,766.67,300,25
61,75,2000,133.33,366.67,10.27,13.33,200,6.67,66.67,83.33,433.33,5.5,766.67,300,25


In [15]:
#Load kcal requirments from FDA

calorieNeeds_female = pd.read_excel('../../FDA Tables XLSX/Appendix 2. Estimated Calorie Needs -2020 Dietary Guidelines _ health.gov_0002.xlsx', sheet_name = 'Females')

calorieNeeds_female.head()

,AGE,Sedentary,Moderately active,Active
0,2,1000,1000,1000
1,3,1000,1200,1400
2,4,1200,1400,1400
3,5,1200,1400,1600
4,6,1200,1400,1600


In [16]:
#Health Parameters Female

femaleNutrition14to18_dict = {'Protein (Cal)' : 0.2/3,
                           'Carbohydrate (Cal)' : 0.55/3,
                           'Fiber (gr)' : 25.2/3,
                           'Total Sugar (gr)': 40/3, #https://www.livestrong.com/article/333407-how-many-calories-should-i-eat-at-lunch/
                           'Total lipid (Cal)': 0.3/3,
                           'Total Trans Fat (Cal)': .01/3, #https://medlineplus.gov/ency/patientinstructions/000786.htm#:~:text=You%20should%20limit%20saturated%20fat,or%202%20grams%20per%20day.
                           'Total Saturated Fat (Cal)': 0.1/3,
                           'Cholesterol (mg)' : 250/3, #https://www.healthline.com/health/high-cholesterol/rda#:~:text=Following%20dietary%20guidelines%2C%20doctors%20used,high%20risk%20of%20heart%20disease.
                           'Calcium (mg)': 1300/3,
                           'Iron (mg)': 15/2,
                           'Sodium, (mg)': 2300/3,
                           'Vitamin A (mcg RAE)': 700/3,
                            'Vitamin C (mg)' : 65/3}
    
femaleNutrition19to30_dict = {'Protein (Cal)' : 0.225/3,
                           'Carbohydrate (Cal)' : 0.55/3,
                           'Fiber (gr)' : 28/3,
                           'Total Sugar (gr)': 40/3, #https://www.livestrong.com/article/333407-how-many-calories-should-i-eat-at-lunch/
                           'Total lipid (Cal)': 0.176/3,
                           'Total Trans Fat (Cal)': .01/3, #https://medlineplus.gov/ency/patientinstructions/000786.htm#:~:text=You%20should%20limit%20saturated%20fat,or%202%20grams%20per%20day.
                           'Total Saturated Fat (Cal)': 0.1/3,
                           'Cholesterol (mg)' : 250/3, #https://www.healthline.com/health/high-cholesterol/rda#:~:text=Following%20dietary%20guidelines%2C%20doctors%20used,high%20risk%20of%20heart%20disease.
                           'Calcium (mg)': 1000/3,
                           'Iron (mg)': 18/2,
                           'Sodium, (mg)': 2300/3,
                           'Vitamin A, IU': 700/3,
                            'Vitamin C (mg)' : 75/3}
    
femaleNutrition31to50_dict = {'Protein (Cal)' : 0.225/3,
                           'Carbohydrate (Cal)' : 0.55/3,
                           'Fiber (gr)' : 25.2/3,
                           'Total Sugar (gr)': 40/3, #https://www.livestrong.com/article/333407-how-many-calories-should-i-eat-at-lunch/
                           'Total lipid (Cal)': 0.176/3,
                            'Total Trans Fat (Cal)': .01/3, #https://medlineplus.gov/ency/patientinstructions/000786.htm#:~:text=You%20should%20limit%20saturated%20fat,or%202%20grams%20per%20day.
                           'Total Saturated Fat (Cal)': 0.1/3,
                           'Cholesterol (mg)' : 250/3, #https://www.healthline.com/health/high-cholesterol/rda#:~:text=Following%20dietary%20guidelines%2C%20doctors%20used,high%20risk%20of%20heart%20disease.
                           'Calcium (mg)': 1000/3,
                           'Iron (mg)': 18/2,
                           'Sodium (mg)': 2300/3,
                           'Vitamin A (mcg RAE)': 700/3,
                            'Vitamin C (mg)' : 75/3}
    
femaleNutrition51_dict = {'Protein (Cal)' : 0.225/3,
                        'Carbohydrate (Cal)' : 0.55/3,
                        'Fiber (gr)' : 22.4/3,
                        'Total Sugar (gr)': 40/3, #https://www.livestrong.com/article/333407-how-many-calories-should-i-eat-at-lunch/
                        'Total lipid (Cal)': 0.176/3,
                        'Total Trans Fat (Cal)': .01/3, #https://medlineplus.gov/ency/patientinstructions/000786.htm#:~:text=You%20should%20limit%20saturated%20fat,or%202%20grams%20per%20day.
                        'Total Saturated Fat (Cal)': 0.1/3,
                        'Cholesterol (mg)' : 250/3, #https://www.healthline.com/health/high-cholesterol/rda#:~:text=Following%20dietary%20guidelines%2C%20doctors%20used,high%20risk%20of%20heart%20disease.
                        'Calcium (mg)': 1200/3,
                        'Iron (mg)': 8/2,
                        'Sodium (mg)': 2300/3,
                        'Vitamin A (mcg RAE)': 700/3,
                        'Vitamin C (mg)' : 75/3}
     

In [17]:
femaleInfo = []

for lifeStyle in ['Sedentary','Moderately active','Active']:
    
    lifeStyle_df = calorieNeeds_female[['AGE', lifeStyle]]
    
    for index,row in lifeStyle_df.iterrows():

        if row['AGE'] >= 14 & row['AGE'] <= 18 :

            for key in femaleNutrition14to18_dict.keys():

                if key not in lifeStyle_df.columns:

                    lifeStyle_df[key] = ''

                    if (key == 'Protein (Cal)') |( key == 'Carbohydrate (Cal)') | (key == 'Total lipid (Cal)')| (key == 'Total Trans Fat (Cal)')|( key == 'Total Saturated Fat (Cal)'):

                        lifeStyle_df.loc[index, key] = round(row[lifeStyle] * femaleNutrition14to18_dict[key],2)

                    else:

                        lifeStyle_df.loc[index, key] = round(femaleNutrition14to18_dict[key],2)

                else:

                    if (key == 'Protein (Cal)') |( key == 'Carbohydrate (Cal)') | (key == 'Total lipid (Cal)')| (key == 'Total Trans Fat (Cal)')|( key == 'Total Saturated Fat (Cal)'):

                        lifeStyle_df.loc[index, key] = round(row[lifeStyle] * femaleNutrition14to18_dict[key],2)

                    else:

                        lifeStyle_df.loc[index, key] = round(femaleNutrition14to18_dict[key],2)

        elif row['AGE'] > 19 & row['AGE'] <= 30 :

            for key in femaleNutrition19to30_dict.keys():

                    if (key == 'Protein (Cal)') |( key == 'Carbohydrate (Cal)') | (key == 'Total lipid (Cal)')| (key == 'Total Trans Fat (Cal)')|( key == 'Total Saturated Fat (Cal)'):

                        lifeStyle_df.loc[index, key] = round(row[lifeStyle] * femaleNutrition19to30_dict[key],2)

                    else:

                        lifeStyle_df.loc[index, key] = round(femaleNutrition19to30_dict[key],2)

        elif row['AGE'] > 31 & row['AGE'] <= 50 :

            for key in femaleNutrition31to50_dict.keys():

                    if (key == 'Protein (Cal)') |( key == 'Carbohydrate (Cal)') | (key == 'Total lipid (Cal)')| (key == 'Total Trans Fat (Cal)')|( key == 'Total Saturated Fat (Cal)'):

                        lifeStyle_df.loc[index, key] = round(row[lifeStyle] * femaleNutrition31to50_dict[key],2)

                    else:

                        lifeStyle_df.loc[index, key] = round(femaleNutrition31to50_dict[key],2)

        elif row['AGE'] > 31 & row['AGE'] <= 50 :

            for key in femaleNutrition51_dict.keys():

                    if (key == 'Protein (Cal)') |( key == 'Carbohydrate (Cal)') | (key == 'Total lipid (Cal)')| (key == 'Total Trans Fat (Cal)')|( key == 'Total Saturated Fat (Cal)'):

                        lifeStyle_df.loc[index, key] = round(row[lifeStyle] * femaleNutrition51_dict[key],2)

                    else:

                        lifeStyle_df.loc[index, key] = round(femaleNutrition51_dict[key],2)
                        
    femaleInfo.append(lifeStyle_df)
                        
femaleInfo[0] #List of DataFrames for males and the intake limits for each meal. 

,AGE,Sedentary,Protein (Cal),Carbohydrate (Cal),Fiber (gr),Total Sugar (gr),Total lipid (Cal),Total Trans Fat (Cal),Total Saturated Fat (Cal),Cholesterol (mg),Calcium (mg),Iron (mg),"Sodium, (mg)",Vitamin A (mcg RAE),Vitamin C (mg)
0,2,1000,66.67,183.33,8.4,13.33,100,3.33,33.33,83.33,433.33,7.5,766.67,233.33,21.67
1,3,1000,66.67,183.33,8.4,13.33,100,3.33,33.33,83.33,433.33,7.5,766.67,233.33,21.67
2,4,1200,80,220,8.4,13.33,120,4,40,83.33,433.33,7.5,766.67,233.33,21.67
3,5,1200,80,220,8.4,13.33,120,4,40,83.33,433.33,7.5,766.67,233.33,21.67
4,6,1200,80,220,8.4,13.33,120,4,40,83.33,433.33,7.5,766.67,233.33,21.67
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,72,1600,106.67,293.33,8.4,13.33,160,5.33,53.33,83.33,433.33,7.5,766.67,233.33,21.67
71,73,1600,106.67,293.33,8.4,13.33,160,5.33,53.33,83.33,433.33,7.5,766.67,233.33,21.67
72,74,1600,106.67,293.33,8.4,13.33,160,5.33,53.33,83.33,433.33,7.5,766.67,233.33,21.67
73,75,1600,106.67,293.33,8.4,13.33,160,5.33,53.33,83.33,433.33,7.5,766.67,233.33,21.67


In [18]:
#Stores the DataFrames in the system to be abe to read it in another Notebook.

%store maleInfo

%store femaleInfo

%store foodFiltered_df

Stored 'maleInfo' (list)
Stored 'femaleInfo' (list)
Stored 'foodFiltered_df' (DataFrame)
